# Recurrent Neural Network Play Generator
We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.

In [2]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

###Dataset
For this example, we only need one peice of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


### Loading own data

In [4]:
#from google.colab import files
#path_to_file = list(files.upload().keys())[0]

### Read contents of File


In [5]:
# Read and then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



## Encoding
Since the text is not encoded , we are going to encode each unique character as different integer

In [7]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
# verify how encoding is done
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [11]:
# reverse function to convert numeric values to text
def int_to_text(ints):
  #trying to creat integet to array if its not in array form
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [12]:
seq_length = 100 # length of sequence for a training example
examples_per_epoch = len(text)// (seq_length+1)

# create entire text stream into dataset
char_dataset =tf.data.Dataset.from_tensor_slices(text_as_int)

In [14]:
# using batch method to turn this stream of characters into batch of desired length
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

In [15]:
# use these sequences of length 101 and split them into input and output
def split_input_target(chunk): # for example : hello
    input_text = chunk[:-1] # hell
    target_text = chunk[1:] # ello
    return input_text, target_text # hell, ello

dataset = sequences.map(split_input_target) # we use map to apply the above function to every entry

In [16]:
for x,y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))




EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [17]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


###Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.

In [19]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [20]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 6.9108773e-03  5.3033661e-03  1.3440752e-03 ... -2.6937143e-03
    1.3620747e-03 -3.2794173e-04]
  [ 3.4022448e-03  6.7274896e-03 -1.2276853e-03 ... -2.8033666e-03
   -9.6223084e-05 -3.3666911e-03]
  [ 3.4799108e-03  9.6073486e-03 -1.4614845e-03 ... -9.2128711e-04
   -3.8834573e-03 -2.1346174e-03]
  ...
  [ 5.9287301e-03  6.2836204e-03  7.7166529e-03 ...  2.3218822e-03
    1.3529011e-03 -1.0486310e-03]
  [ 7.7687418e-03  6.3430215e-03  1.1377629e-02 ...  6.9505093e-04
    4.3713762e-03  1.8154928e-03]
  [ 4.1177724e-03  1.8239446e-03  7.1210391e-03 ... -5.3840561e-04
    3.2439232e-03 -2.4989811e-03]]

 [[ 1.0422568e-03  4.8418180e-03 -1.6369161e-03 ...  2.2528722e-04
   -3.5355412e-03  4.0890020e-04]
  [ 1.2392149e-04  4.8133461e-03  2.4328893e-03 ...  1.2697529e-03
   -4.9902978e-03  4.5902832e-04]
  [-4.8546994e-04  5.6736926e-03 -3.9585978e-03 ... -1.2896729e-03
   -4.1531594e-03 -6.8601518e-04]
  ...
  [ 2.8872332e-03  1.1626582e-02 -1.1876889e-02 ... -2.4883370e

In [21]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 6.9108773e-03  5.3033661e-03  1.3440752e-03 ... -2.6937143e-03
   1.3620747e-03 -3.2794173e-04]
 [ 3.4022448e-03  6.7274896e-03 -1.2276853e-03 ... -2.8033666e-03
  -9.6223084e-05 -3.3666911e-03]
 [ 3.4799108e-03  9.6073486e-03 -1.4614845e-03 ... -9.2128711e-04
  -3.8834573e-03 -2.1346174e-03]
 ...
 [ 5.9287301e-03  6.2836204e-03  7.7166529e-03 ...  2.3218822e-03
   1.3529011e-03 -1.0486310e-03]
 [ 7.7687418e-03  6.3430215e-03  1.1377629e-02 ...  6.9505093e-04
   4.3713762e-03  1.8154928e-03]
 [ 4.1177724e-03  1.8239446e-03  7.1210391e-03 ... -5.3840561e-04
   3.2439232e-03 -2.4989811e-03]], shape=(100, 65), dtype=float32)


In [22]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 6.9108773e-03  5.3033661e-03  1.3440752e-03 -5.5161798e-03
  1.8172832e-04  1.1176679e-03 -4.3828958e-03 -8.2184735e-04
  1.9622257e-03 -1.4475846e-03 -5.2393135e-03 -3.7555627e-03
  6.2909268e-04  3.1934071e-03  7.0470414e-04 -5.0566411e-03
 -4.0326733e-03 -5.2320550e-04  3.4645125e-03 -1.1724551e-03
 -3.2369313e-03  2.1221554e-03  1.0838794e-03 -3.1440181e-04
  7.0579164e-04  1.2543518e-03  1.0887710e-03 -2.4445711e-03
 -1.0187144e-04  3.3791750e-03  3.3447412e-03  1.6005801e-03
  5.0645778e-03 -3.2322740e-03 -4.2445280e-04  4.4620247e-03
 -7.1328366e-04  7.6922763e-04  8.7235862e-04  7.6370677e-03
 -4.8498274e-05  1.2820244e-03  4.0964660e-04 -9.0387918e-04
  1.7341546e-03 -4.4516986e-04 -2.6031453e-03  1.1000434e-03
 -3.8502708e-03  1.2588131e-03  4.4594645e-03  5.5959756e-03
 -3.4557568e-04 -3.1046721e-04 -2.0011838e-03  6.0208584e-04
  1.6860325e-03  3.0052941e-03  1.1122103e-03 -1.6212420e-03
 -2.6269946e-03 -2.3308371e-03 -2.6937143e-03  1.3620747e-03
 -3.279417

In [23]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"K;pKW,vgvN'WDV-p?qeNP;L.EGJurqKh'KRlM\nChdfhKNlH;Y'S:Jfgrd$Tu3mJqgAKqLLWVqR3dSs;v;aIIlhEV$jWxLGtaBEv!"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next.

In [25]:
model.compile(optimizer='adam', loss=loss)

###Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [28]:
# Training

history = model.fit(data, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 13s 68ms/step - loss: 0.9046
Epoch 2/10
172/172 [==============================] - 13s 69ms/step - loss: 0.8653
Epoch 3/10
172/172 [==============================] - 13s 70ms/step - loss: 0.8264
Epoch 4/10
172/172 [==============================] - 13s 69ms/step - loss: 0.7910
Epoch 5/10
172/172 [==============================] - 13s 69ms/step - loss: 0.7565
Epoch 6/10
172/172 [==============================] - 14s 70ms/step - loss: 0.7239
Epoch 7/10
172/172 [==============================] - 13s 68ms/step - loss: 0.6943
Epoch 8/10
172/172 [==============================] - 13s 68ms/step - loss: 0.6670
Epoch 9/10
172/172 [==============================] - 13s 68ms/step - loss: 0.6406
Epoch 10/10
172/172 [==============================] - 13s 69ms/step - loss: 0.6206


###Loading the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.

In [29]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model has finished training, we can find the **lastest checkpoint** that stores the models weights using the following line.

In [30]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

###Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [32]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: i go for morning walk to park near to my house
i go for morning walk to park near to my house,
That you notoly office and to yield!

AEdile:
He's coming;
And many strokes, they'll go along with thee,
Justee, I know not what I did but speak
And bid him call the heir of this apparel,
That e'er I wish'd of that, my due by with an
fancias on wine. But what say you to me?

BIONDELLO:
Force and holy loverce!

PRINCE EDWARD:
A blessed, bloody, are thy hands, and thou takest for player
so to our trestory.

PETRUCHIO:
Sirrah young affection lack their griefs,
To visit you where you stay to wripe up,
And this same place, take't.

BIONDELLO:
Tainth!
Speak to my side: come by my true love's requirition, officence, and I mean
For 'tis no chat.

PETRUCHIO:
The very wreck me not, I dispose of him.
May now tend the world of victory,
Like voices that he doth quicken him.

MENENIUS:
He's remorse; t
